In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read Data

In [ ]:
df=pd.read_csv('../input/gender-classification/Transformed Data Set - Sheet1.csv')

# Initial Data Analysis

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

all 4 features are object type and there is no null values in the data set. so we need to go with onehot encoding to convert the categorical values.

# Importing plotting libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Plot all columns using SNS countplot.

In [ ]:

fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(10,18))
for i in range(len(df.columns)):
    sns.countplot(data=df, x=df.iloc[:,i],ax=axes[i])
    

In [ ]:
len(df)

# Train Test Split

In [ ]:
X=df.drop('Gender',axis=1)
y=df['Gender']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
cols=list(X.columns)

In [ ]:
cols

# Encode categorical values

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot= OneHotEncoder()
from sklearn.compose import ColumnTransformer
clt = ColumnTransformer([('binarize',onehot,cols)], remainder='passthrough')

In [ ]:
X_train=clt.fit_transform(X_train)

In [ ]:
X_test=clt.transform(X_test)

# Creating model & metrics

# KNeighboursClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
test_score=[]
for i in range(1,30):
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_hat=knn.predict(X_test)
    score= 1-accuracy_score(y_test,y_hat)
    test_score.append(score)

Lets us first evaluate with KNeighboutClassifier with different n_neighbour values

In [ ]:
plt.plot(test_score)

In [ ]:
knn.get_params()

Lets do the crossvalidation using GridSearchCV with KNN

# KNN with GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
n_val=list(range(1,40))
param_grid={'n_neighbors':n_val}
model = GridSearchCV(knn,param_grid=param_grid, cv=10, scoring='accuracy')

In [ ]:
model.fit(X_train,y_train)

Best Parameters in KNN using GridSearchCV

In [ ]:
model.best_estimator_.get_params()

In [ ]:
grid_cv_predict = model.predict(X_test)

In [ ]:
accuracy_score(y_test,grid_cv_predict)

How ever the Accuracy score for the model is 54%, so lets try with different models

In [ ]:
plt.figure(figsize=(18,10))
pd.DataFrame(model.cv_results_)['mean_test_score'].plot()
plt.title('Mean Test score using KNN')

# Logistic Regression - Cross Validation

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
logit = LogisticRegressionCV()
logit.fit(X_train,y_train)

In [ ]:
Logit_pred=model.predict(X_test)
accuracy_score(y_test,Logit_pred)

We are getting the same values as KNN

# # Support Vector Machine - Classification

In [ ]:
from sklearn.svm import SVC
svc =SVC()
svc.fit(X_train,y_train)
pred_svc = svc.predict(X_test)
accuracy_score(y_test,pred_svc)

Accuracy score is better compared to other 2 models. 

In [ ]:
print(classification_report(y_test,pred_svc))

In [ ]:
sns.heatmap(confusion_matrix(y_test,pred_svc), annot=True)

# Conclusion
Reason for the low accuracy:
1. Dataset is small - 66 rows
2. we splited the data set with Test & Train datasets.